---
title: "Computer Graphics"
author: "Vahram Poghosyan"
date: "2025-05-24"
categories: ["Game Development", "Computer Graphics"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js",
              "OrbitControls": "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js",
              "Cannon": "https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-1.js"></script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-2.js"></script>
---

# OpenGL/WebGL & Shaders

OpenGL is a cross-platform graphics API that is widely used in video game development and computer graphics (it's a competitor to Nvidia's DirectX). WebGL is a [language binding](https://en.wikipedia.org/wiki/Language_binding) of OpenGL in JavaScript. I will use WebGL and OpenGL interchangeably throughout these posts. It provides a set of functions for rendering 2D and 3D graphics, allowing developers to create complex visual effects and realistic environments. GLSL is part of OpenGL. In the Three.js post [Three.js: Introduction to 3D Graphics](../../visualization/three_js_in_jupyter/three_js_in_jupyter.ipynb), we learned how to create a simple 3D scene using Three.js, which is a JavaScript library that abstracts the use of OpenGL API. It uses OpenGL's API calls, internally, to compile, link, and send our GLSL shader code to the GPU.

In this post, we will explore how to use shaders in Three.js to create custom visual effects. Shaders are small programs that run on the GPU and are used to control the rendering of graphics. They allow developers to manipulate the appearance of objects in a scene, such as their color, texture, and lighting. Shaders are separate from the [*rendering pipeline*](#the-three-stages-of-the-rendering-pipeline) (see below), they can be thought of as ad-hoc programs that run on the GPU in a massively parallel way. They can run at *any* point of the rendering pipeline, and do so totally independently of it. Because of this, shaders give us fine-grained control over the rendering process.

Let's create a new scene containing a simple quadrilateral mesh (created using Three.js, which is an abstraction layer over OpenGL). We will use this quad mesh as our canvas to draw on (with shaders). First, we'll stick to 2D. We'll create a vertex shader that takes the vertices of our quadrelateral and maps them to the full screen. The fragment shader will contain most of the magic, for now. It will essentially paint the surface of the quad... 

<details><summary>Click to expand the code used to create a basic scene</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm';

/* Scene / Camera / renderer ---------------------------------------------- */ 
const bodyWidth = document.getElementById("quarto-document-content").clientWidth;
const bodyHeight = 600;
const canvas = document.getElementById("three-d-canvas");

const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

/* Lights ---------------------------------------------- */
scene.add(new three.AmbientLight(0xffffff, 1));
const dir = new three.DirectionalLight(0xffffff, 1);
dir.position.set(10, 20, 10);
scene.add(dir);

/* Plane  ---------------------------------------------- */
const quadMesh = new three.Mesh(
    new three.PlaneGeometry(250, 250), // Plane is added to the XY plane (x+ = right, y+ = up, z+ = out of the screen)
);
scene.add(quadMesh); 

/* Camera ---------------------------------------------- */
camera.position.set(0, 0, 200); // The z+ represetns 100 units towards the viewer (out of the screen)
camera.lookAt(quadMesh.position);

renderer.render(scene, camera);
```

</details>

This is pretty much it for the CPU-side code that creates the scene, camera, and renderer. The next step is to create the shaders.

Final Result:

In [1]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-1'></canvas>"))

Here it is, our empty canvas. Let's add some color to it. 

## Shader Code (GLSL)

As mentioned above, the shader code is written in GLSL (OpenGL Shading Language). The *vertex shader* is responsible for transforming the vertices of the geometry. The fragment shader is responsible for determining the color of each pixel. Note that the fragment shader is also sometimes called *the pixel shader*.

First, we create a vertex shader in the `/shaders` subdirectory of this page. Then, a fragment shader. We import these shaders into our JavaScript code and use them to create a `ShaderMaterial`. Finally, we apply this material to our quad mesh.

```js
import vertexShader from 'https://raw.githubusercontent.com/vahram-pg/digital-garden/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.vert?raw';
import fragmentShader from 'https://raw.githubusercontent.com/vahram-pg/digital-garden/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.frag?raw';
const shaderMaterial = new three.ShaderMaterial({
    vertexShader: vertexShader,
    fragmentShader: fragmentShader,
    uniforms: {
        uTime: { value: 0.0 },
        uResolution: { value: new three.Vector2(bodyWidth, bodyHeight) }
    }
});

quadMesh.material = shaderMaterial;
```

Note, if we add empty shaders to an object in our canvas we will see pitch black and the following error will be printed to the console:

```
Program Info Log: Vertex shader is not compiled.
```
This is expected, because we haven't written any code in our vertex shader yet. Let's code up our shaders! 

### GLSL Basics

#### Main Function

Every shader has a main function, which is the entry point for the shader code. We can write more functions, which main will call.

```glsl
void otherFunction() {
    // More code here
}

void main() {
    // Your code here
}
```
#### Types

GLSL is strongly typed, meaning we need to declare the types of our variables. The most common types are `float`, `int`, `vec2`, `vec3`, and `vec4`.

- A `float` variable needs to explicitly have a decimal point, e.g. `1.0` or `3.14` or `2.` (otherwise it's interpreted as an `int`).
- All `vec` types are vectors, which are arrays of floats. For example, `vec2` is a 2D vector (array of 2 floats). Vectors can be initialized using the `vec2(x, y)` constructor, where `x` and `y` are the components of the vector. Initializing based on one parameter will create a vector with all components equal to that parameter, e.g. `vec2(1.0)` will create a vector with both components equal to `1.0`.

#### Reading Vectors

We can read the components of a vector using the `x`, `y`, `z`, and `w` properties (`w` is the translation in homogeneous coordinates). If the vector represents a color, we can also use `r`, `g`, `b`, and `a` properties (where `a` is the opacity value). 

Given the vector `vec4 v = vec4(0.1,0.2,0.3,0.4)`, we can access its components as follows:

| 0.1 | 0.2 | 0.3 | 0.4 |
|-----|-----|-----|-----|
| `v.x`   | `v.y`   | `v.z`   | `v.w`   |
| `v.r`   | `v.g`   | `v.b`   | `v.a`   |
| `v.s`   | `v.t`   | `v.p`   | `v.q`   |

We can also access multiple components at once using the `xy`, `xyz`, and `xyzw` properties. For example, `v.xy` will return a `vec2` with the first two components of `v`, i.e. `vec2(0.1, 0.2)`. This is called vector *swizzling*.

Swizzling use cases include:
- Extracting a 2D vector from a 3D vector (such as for simple projection), e.g. `vec3 v = vec3(1.0, 2.0, 3.0); vec2 v2 = v.xy;` will create a `vec2` with the first two components of `v`.
- Swapping color channels, e.g. `vec4 color = vec4(1.0, 0.0, 0.0, 1.0); vec4 swappedColor = color.bgra;` will create a new color with the blue channel in the first position, green in the second, red in the third, and alpha in the fourth.

This is a very powerful way to go hop from dimension into dimension, and to manipulate vectors in a very flexible way. Say we wanted to go from 2D to 3D and we didn't care about the z-coordinate, we could do this:

```glsl
vec2 v1 = vec2(1.0, 2.0);
vec3 v2 = v1.xyx // This will initialize a 3D vector with its z-coordinate equal to the x-coordinate of the 2D vector.
```


#### Attributes, Uniforms, and Varying

Just as we can instantiate variables inside our GLSL shader code, some are also passed into our shaders from the external OpenGL context. These are called *attributes*, and *uniforms*.

We've already seen how to pass uniforms into our shaders using Three.js's `ShaderMaterial`. Here's a quick recap:

```js
const shaderMaterial = new three.ShaderMaterial({
    vertexShader: vertexShader,
    fragmentShader: fragmentShader,
    uniforms: {
        uTime: { value: 0.0 },
        uResolution: { value: new three.Vector2(bodyWidth, bodyHeight) }
    }
});
```
The above code passes two uniforms into our shaders: `uTime` and `uResolution`. These can be used in both the vertex and fragment shaders to control the rendering process. For example, `uTime` can be used to create animations, while `uResolution` can be used to adjust the rendering based on the size of the HTML canvas.

The difference between attributes and uniforms is that attributes are per-vertex data, while uniforms are global data that is shared across all vertices and fragments. See table below for a summary of the differences:

|                                  | Attribute      | Uniform     |
|----------------------------------|----------------|-------------|
| Available in Vertex Shader?      | Read Only      | Read only   |
| Available in Fragment Shader?    | N.A            | Read only   |
| Set From                         | CPU            | CPU         |
| Contain Information              | Per Vertex     | Constant    |

There are also *varying* variables, which are used to pass data from the vertex shader to the fragment shader (because they're the only variables that are writable to). Varying variables are written to inside the vertex shader and passed as read-only to the fragment shader. They are commonly used for interpolating values across the surface of a mesh. For example, if we want to pass the color of a vertex to the fragment shader, we can declare a varying variable in the vertex shader and assign it a value. Then, in the fragment shader, we can read that varying variable to get the interpolated color for each pixel.

Here's the table summarizing the differences between attributes, uniforms, and varying variables:

|                                  | Attribute   | Uniform     | Varying       |
|----------------------------------|-------------|-------------|---------------|
| Available in Vertex Shader?      | Read Only   | Read only   | Read/Write    |
| Available in Fragment Shader?    | N.A         | Read only   | Read Only     |
| Set From                         | CPU         | CPU         | Vertex Shader |
| Contain Information              | Per Vertex  | Constant    | Per Fragment  |

Attributes contain information per vertex (i.e. data about each vertex, such as its color) because they're passed to the vertex shader as variables (which works on vector geometry -- i.e. vertices). Varying Variables, on the other hand, are used to pass data from the vertex shader down to the fragment shader, which works with fragments, hence they contain information *per fragment*.

#### Normalized Vectors

In GLSL, vectors are often normalized to have a length of 1. This is useful for many operations, such as lighting calculations and texture mapping. Normalizing a vector is done using the `normalize()` function. For example, if we have a vector `vec3 v = vec3(1.0, 2.0, 3.0);`, we can normalize it by calling `vec3 normalizedV = normalize(v);`. This will create a new vector with the same direction as `v`, but with a length of 1.

Individual entries are normalized too. For example white is represented as `vec3(1.0, 1.0, 1.0)`, rather than `vec3(255, 255, 255)`. Note, this vector is not normalized. But *normalization* is also often used in this sense.

### Writing Our First Basic Shaders

Let's start with a fragment shader that maps all pixels to a single color. The function `main()` below runs per each pixel and sets their value to purple. We use this to simply color the entire quad purple.

```glsl
void main() {
    // Make every pixel purple
    gl_FragColor = vec4(0.5, 0.0, 0.5, 1.0); // Set the color to purple (RGBA)
}
```

Now, let's write our first vertex shader. It will simply stretch out the rectangle to cover the entire screen. Then, we will pass the final vertex positions to the fragment shader, which will use them to color the pixels.

As mentioned before, the vertex shader receives attributes (per vertex data). These include:

```glsl
attribute vec3 aPosition;
attribute vec2 aTexCoord;
```

- `aPosition` is the position (relative to world space) of the vertex in 3D space (x, y, z). Our vertex shader will run once per vertex, and it will receive the position of that vertex as an attribute.
- `aTexCoord` is the texture coordinate of the vertex (u, v). This is used to map textures onto the geometry in the fragment shading stage. This is the coordinate we would use in the fragment shader to draw on the pixel corresponding to this vertex.

These attributes are automatically set by Three.js (via the OpenGL API, internally).

In our vertex shader we define a Varying Variable `pos`, which will be used to pass the texture coordinates (`aTexCoord`) to the fragment shader. Essentially, after the vertex shader is done scrambling the vertices, the new texture coordinates of these scrambled vertices are passed down to the fragment shader.

Let's stretch the quad to cover the entire screen.

<details><summary>Click to expand the vertex shader code</summary>

```glsl
attribute vec3 aPosition;
attribute vec2 aTexCoord;

varying vec2 pos;

void main() {
    pos = aTexCoord;

    vec4 position = vec4(aPosition, 1.0);
    position.xy = position.xy * 2.0 - 1.0; // Map the positions from the [0,1] range to the [-1,1] range

    gl_Position = position;
}
```

</details>


This gives:

In [2]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-2'></canvas>"))

# The Main Stages of the Rendering Pipeline

After some CPU side pre-processing, that breaks our scene down into vertex data and sends it to the GPU, the rendering pipeline consists of three main stages:

## Vertex Shading

The first stage of the rendering pipeline is the vertex shading stage. In this stage, the vertex shader is executed for each vertex of the geometry. The vertex shader is responsible for transforming the vertices of the geometry from object space to clip space. It can also be used to perform other operations, such as calculating normals or texture coordinates.

```mermaid
graph TD
    A["Vertex (x, y, z),(u, v),(n1, n2, n3),id" ] -->|Input| B[Vertex Shader]
    C["Matrix (Model/View/Projection)" ] -->|Uniform| B
    B -->|Output| D[Transformed Vertex]
```

### World Matrix, Local Matrix, and Model Matrix

This is where three crucial matrices come into play: the *model matrix*, the *view matrix*, and the *projection matrix*. The model matrix transforms the vertices from object space to world space, the view matrix transforms the vertices from world space to camera space, and the projection matrix transforms the vertices from camera space to clip space. The final position of each vertex is calculated by multiplying these three matrices together. As always with matrices, the order of multiplication matters. The final position of each vertex is given by:

The position of a given vertex $\mathbf{v}$ in clip space is:

$$
\mathbf{v}_{\text{clip}} = \mathbf{P} \cdot \mathbf{V} \cdot \mathbf{M} \cdot \mathbf{v}
$$

Once the position of each vertex is calculated, a *primitives assembly* stage is performed, where the vertices are grouped into primitives (e.g., triangles, lines, etc.). The primitives are then rasterized to generate fragments, which are the pixels that will be drawn on the screen.

Geometry shading is a more advanced stage that can be used to generate new primitives based on the existing primitives. It is not always used, but it can be useful for certain effects, such as generating shadows or reflections.

A *tessellation stage* can also be used to subdivide the primitives into smaller ones, allowing for more detailed rendering. This is often used in high-end graphics applications, such as video games and simulations.

### UV Coordinates

A UV coordinate is simply a 2D point on a texture atlas (or image) that tells the GPU “for this vertex, here’s where you should sample the texture.” Think of it like this: Our mesh lives in 3D space: each vertex has a position $(x, y, z)$, a normal vector, maybe a color, etc.
If we want to “paint” an image onto that mesh -— a brick wall, a wood plank, whatever -— we need a way to say “vertex $v$ in 3D should use the pixel at $(u, v)$ in my 2D bitmap.” That $(u, v)$ pair is just two numbers, usually in the range $[0, 1]$, that index into the texture. $(0, 0)$ is, by convention, the bottom-left corner of the texture; $(1, 1)$ is top-right. Any point in between picks a pixel (or a filtered blend) in the middle.

UV coordinates are per-vertex attributes, so they’re defined in the vertex shader only. The fragment shader, which runs per pixel of a fragment, gets an interpolated UV coordinate. Once the GPU has processed all vertices of a triangle, it figures out which pixels (*fragments*) lie inside that triangle, and runs the fragment shader for each of those pixels. By construction, a fragment shader does not have direct access to *attribute* inputs -— those only live in the vertex stage. A triangle face has three corners (three vertices), each corner has its own $(u, v)$. When the rasterizer draws all the pixels inside that triangle (more on this in the section right below this one), it “interpolates” between those three $(u, v)$-s so that each pixel (fragment) ends up sampling the exact correct spot on the texture.

In our vertex shader, we write something like:

```glsl
out vec2 vUv; // A varying that stores the UVs of the vector to pass along to the fragment shader

void main() {
    // Compute view direction in world space
    vec4 worldPos = modelViewMatrix * vec4(position, 1.0) ;
    vec3 viewDir = normalize(-worldPos.xyz) ;
    // Output vertex position
    gl_Position = projectionMatrix * worldPos;
    // Pass the UV coordinates to the fragment shader
    vUv = uv;
}
```

This vertex shader runs per-vertex, and it outputs the UV coordinates of each vertex to the fragment shader via a varying variable `vUv`. The fragment shader can then use this varying variable to sample the texture at the correct location for each pixel that resides in the primitive it's part of (as determined by the rasterization step). Each pixel (i.e. fragment) then has its individual UV coordinate, which is interpolated from the UV coordinates of the vertices of the primitive it belongs to.

## Rasterization and/or Ray-Tracing

The magical rendering pipeline stage where vector-based primitives are converted into fragments (final abstraction layer before pixels, basically the 2D projection on the screen of the 3D primitives). The rasterization stage takes the primitives (generated in the previous stage) and converts them into fragments, which are then shader by the fragment shader (the next stage of the rendering pipeline) to produce the actual pixels values that will be shown to the screen. [Rasterization](https://en.wikipedia.org/wiki/Rasterisation) and [ray-tracing](https://en.wikipedia.org/wiki/Ray_tracing_(graphics)) achieve this same exact end goal using very different approaches. Both algorithms use *lighting models* (like [Phong, or Blinn-Phong](https://en.wikipedia.org/wiki/Blinn%E2%80%93Phong_reflection_model) for *diffuse light*, and [Lambert](https://en.wikipedia.org/wiki/Lambertian_reflectance) for *specular highlights*) to calculate the final value of a pixel based on the light sources that exist in the scene, but ray-tracing does so by actually simulating the path of light rays in the scene. Rasterization, on the other hand, does so by interpolating the values of the vertices of the primitive that the pixel belongs to (since a lot of information is associated with each vertex, including its position in space, its color, its texture and its *normal*). Rasterization also applies lighting effects by using lighting models and the known positions (and directions) of the light sources. A depth or *z-buffer* is also used to store pixel depth information to ensure front-most objects at a pixel’s $(x,y)$ screen location are displayed on-screen, and objects behind the front-most object remain hidden. So, rasterization relies heavily on vertex-to-pixel interpolation. Ray-tracing, on the other hand, simulates light transport and performs exact calculations (rather than interpolation). It also doesn't need to use a z-buffer as rays will naturally intersect with the closest primitive in the scene. 

Note that while ray-tracing calculations need to be performed in 3D space, ray-tracing is performed at the fragment-level as well, much like rasterization, rather at than at the primitive-level (in the vertex shader). Why? Well, vertex shaders execute once per vertex, but ray-tracing needs to be performed for each pixel on the screen. Ray-tracing also requires random access to the entire scene geometry. Vertex shaders only have access to data for the single vertex being processed, while fragment shaders have access to the entire scene data via uniforms/textures/buffers (which is just the way the rendering pipeline is designed). Not to mention, vertex shaders can't directly write color outputs to arbitrary pixels - they pass data downstream to the fragment shader. Ray-tracing is fundamentally a per-pixel operation (for each pixel, determine what color it should be based on ray intersection tests). 
But how can a ray-tracer work inside a fragment shader? Because the fragment shader gets a bunch of variables passed to it as attributes from upstream of the rendering pipeline (i.e. from the vertex shader or even the 3D framework, or engine, itself) scene data like the camera position are known to the vertex shader. Based on these known values, a surface (albeit a primitive one) can be defined in the fragment shader itself (mathematically). So, the fragment shader may be aware of an object that's defined within it as code, and it can simulate interactions with said object before assigning a value to a particular pixel. This is the basis of ray-tracing inside a fragment shader as we shall see. But this limits us to ray-tracing of scenes comprised of primitive shapes (like spheres, planes, etc.) that can be defined mathematically. For more complex scenes, perhaps with imported geometry, we need to use a different approach. There are pure ray-tracing pipelines which don't use rasterization at all, and provide specialized shader types (ray generation shaders, intersection shaders, closest hit shaders, etc.). We'll learn about these later. There are also *hybrid rendering pipelines* that use rasterization for the bulk of the scene and ray-tracing for advanced visual effects.

Ray-tracing can also be used in conjunction with rasterization to achieve a balance between quality and performance, as well as to render complex scenes with imported geometry. This is the approach taken by most video games today. `WebGLRenderer` (and the `CanvasRenderer`) in Three.js, for example, use a rasterization pipeline. However, since parts of the rasterization pipeline are open to being programmable (with shaders), we can write our own ray-tracing implementation (which is what we'll eventually try to do).

Before now, we only saw a trivial example of a hybrid rendering pipeline. The pipeline rasterized a simple quadrilateral, initially, to cover the whole screen. This was done to, basically, start a computation thread for each pixel where we could do the ray-tracing. But we can also rasterize a good chunk of our scene and use ray-tracing only for certain, advanced visual effects. For example, ray-tracing can be used to calculate reflections and refractions, while rasterization can be used for the rest of the scene. This is often referred to as a *hybrid rendering pipeline*. For example, *screen-space reflections (SSR)* is a technique that uses ray-tracing to calculate reflections in a scene, while rasterization is used for the rest of the scene. This allows for more realistic reflections without the performance overhead of full ray-tracing. 

How does this work? After the first rasterization pass: 

- The scene (including complex imported models) is rasterized into fragments
- A primary visibility buffer (what's directly visible to the camera) is created
- *G-buffers* store additional data like depth, normals, material properties

SSR then ray-marches against the depth buffer.

*Ambient occlusion* is another technique that uses ray-tracing to calculate the amount of light that reaches a surface, while rasterization is used for the rest of the scene. While these can be thought of as *post-processing effects*, but that term can also mean "effects that are applied after the entire scene has been drawn and colored (i.e. after *fragment shading*)." These effects include: *bloom*, *motion blur*, *depth of field*, *aliasing*, etc. However, after rasterization we already have a colored scene, more or less, so the term post-processing can still apply. *Fragment shading* and *post-processing* are basically at the same level in the pipeline, it's just a matter of preference whether to call these effects post-processing or not. *Post-processing* is *fragment shading* (in essence). The 

But why does ray-tracing start from the camera (more specifically the *near-plane* of the camera) and not the object? After all, in the real world, the sun's rays reflect off of objects *into* our eyes (and not vice-versa). Most of the rays cast from objects would not even hit the camera, so doing that will lead us to calculate the path of a lot of extra rays for no reason. *The Principle of Reversibility of Light*, sometimes referred to as the *Principle of Reciprocity* in optics, states that if a ray of light is sent along a path and its direction is reversed, it will retrace its original path exactly. This holds true regardless of reflections or refractions the light ray undergoes. In other words, when we see someone in a mirror they can also see us back. This means casting rays from the camera into the scene is equivalent to casting rays from the scene into the camera.

To further understand the difference between rasterization and ray tracing at an algorithmic level, think in terms of two nested loops. Each primitive covers a certain area of pixels on the screen, so the rasterization algorithm takes each primitive first and determines which pixels it covers. For each primitive, and for each pixel, rasterization asks: does the primitive cover the pixel? Does it fall outside the pixel? Maybe there's another primitive in front of it? Ray tracing, on the other hand, flips the loops. It asks, for each pixel, which primitive is the closest one to it.  It does this by casting *rays* from the camera into the scene and checking for intersections with primitives. 

Here's a great [video](https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s) from Nvidia that explains the difference between rasterization and ray-tracing in more detail. I will embed it here.

{{< video https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s >}}\

Rasterization is the common approach taken by video games (or other real-time applications)because it's computationally cheaper (so a single frame can be rendered in fractions of a second), while ray-tracing is more accurate and more computationally intensive (it's used in Pixar-style animations where render farms can take their time rendering the frames of a movie). However, this is changing with the advent of modern GPUs that can perform ray-tracing in real-time.

### Ray-Tracing vs Ray-Marching

Ray-marching, on the other hand, is also a ray-tracing algorithm that works by casting a ray from the camera of a scene, however, whereas ray-tracing finds the exact intersection of a ray and a primitive, ray-marching iteratively approaches this intersection point. Ray-marching is used for volumetric shapes (like clouds), and to render fractal worlds that have infinite detail (see this cool [video](https://www.youtube.com/watch?v=svLzmFuSBhk))!

{{< video https://www.youtube.com/watch?v=svLzmFuSBhk >}}

Ray-marching is better form fractals because fractals (and other procedural geometry) are implicitly defined surfaces (defined by mathematical functions like SDFs), rather than actual primitives. We can use ray-tracing too, but that would require taking a *mesh approximation* of the fractal (which is an expensive operation that also removes the infinite detail from the fractal). While traditional ray tracing excels at rendering explicit, polygonal geometry by calculating exact intersections, ray marching thrives on implicit, mathematically defined surfaces (like fractals) by iteratively stepping through space using distance information (SDFs, once again).

## Fragment Shading

The final stage of the rendering pipeline is the fragment shading stage. In this stage, the fragment shader is executed for each fragment generated in the rasterization stage. The fragment shader is responsible for determining the color of each pixel based on the lighting and shading calculations performed in the previous stages. It can also be used to apply *textures*, perform *post-processing* effects, and more.

A key object in this stage is the *frame buffer*, which is a memory buffer that stores the color and depth information of each pixel. A *Frame Buffer Object (FBO)* is an object that's used to store the shader calculations on a given scene to a *texture* rather than to the screen itself. This allows us to overlay post-processing effects. Each effect is drawn to its own *texture* within a separate FBO. What's shown on the screen is, then, all of these *textures* (along with other shader calculations performed during the fragment shading stage) overlaid on top of the rasterized scene.

# The Rendering Pipeline

This has to be fact checked, but here's a pipeline diagram.

```mermaid
flowchart TD
    subgraph "CPU Processing"
        A[Scene Data] -->|"Mesh Data\nTextures\nLights\nMaterials"| B[Geometry Processing]
        B -->|"Vertex Data\nIndices\nShader Programs"| C[Send to GPU]
    end
    
    subgraph "GPU Pipeline"
        C -->|"Vertex Attributes\n(positions, normals, UVs)"| D[Vertex Shader]
        
        D -->|"Transformed Vertices"| E[Primitive Assembly]
        
        subgraph "Optional Stages"
            E -->|"Primitives"| F1[Tessellation]
            F1 -->|"Subdivided Primitives"| F2[Geometry Shader]
            F2 -->|"Modified/New Primitives"| G
        end
        
        E -->|"Assembled Primitives\n(triangles, lines)"| G[Rasterization]
        
        G -->|"Fragments\n(potential pixels)"| H[Fragment Shader]
        
        H -->|"Colored Fragments"| I[Per-Sample Operations]
        I -->|"Final Pixel Colors"| J[Frame Buffer]
    end
    
    subgraph "Matrix Transformations"
        M["Model Matrix (M)\nObject → World Space"] -.->|"Uniform"| D
        V["View Matrix (V)\nWorld → Camera Space"] -.->|"Uniform"| D
        P["Projection Matrix (P)\nCamera → Clip Space"] -.->|"Uniform"| D
    end
    
    subgraph "Varying Data Flow"
        D -.->|"Interpolated Data\n(UVs, normals, etc.)"| H
    end
    
    subgraph "Alternative Path"
        G -->|"For ray-tracing effects"| RT[Ray Generation]
        RT -->|"Rays"| RT2[Ray Intersections]
        RT2 -->|"Hit Information"| H
    end
    
    J -->|"Display"| K[Screen]
```    

# Euler Angles and Rotation Matrices

## Intertial Frame and Body Frame

$R_{yaw}$, $R_{pitch}$, and $R_{roll}$.

# Quaternion Rotation